In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
import optuna
from lifelines.utils import concordance_index

import matplotlib.pyplot as plt

2023-05-15 04:17:25,346 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [2]:
SEED = 69

import os
os.environ['PYTHONHASHSEED']=str(SEED)

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

In [3]:
data = pd.read_parquet("data/data_with_feats_4xgb.parquet.gzip")

In [4]:
target_cols = [col for col in data.columns if "target" in col] + ['term']
target_cols

['target', 'target_month', 'target_xgb', 'term']

In [5]:
features = ["annual_inc", "loan_amnt", "fico_average", "emp_length", "acc_open_past_24mths", "avg_cur_bal" , "sub_grade", "int_rate", "revol_util"]

features += [col for col in data.columns if "home_ownership_ohe_" in col]

data_used_cols = data[features + target_cols].dropna()
# assert data.shape[0] == data_used_cols.shape[0]

features

['annual_inc',
 'loan_amnt',
 'fico_average',
 'emp_length',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'sub_grade',
 'int_rate',
 'revol_util',
 'home_ownership_ohe_ANY',
 'home_ownership_ohe_MORTGAGE',
 'home_ownership_ohe_NONE',
 'home_ownership_ohe_OTHER',
 'home_ownership_ohe_OWN',
 'home_ownership_ohe_RENT']

In [6]:
data["target_xgb"].value_counts()

0    1723625
1     133750
Name: target_xgb, dtype: int64

In [7]:
data_true = data_used_cols.loc[data_used_cols["target_xgb"] == 1]
data_false = data_used_cols.loc[data_used_cols["target_xgb"] == 0]

In [8]:
X, y = data_false[features], data_false[target_cols]

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.6, random_state=SEED)

In [9]:
X_train_val = pd.concat([X_train_val, data_true[features]], axis=0)
y_train_val = pd.concat([y_train_val, data_true[target_cols]], axis=0)

In [10]:
(y_train_val["target_xgb"] == 1).sum()

130489

# Predict

In [11]:
def get_proba_for_month(proba, n_months):
    # PDn = 1 - (1-pd12)^(n/12)

    return 1 - (1 - proba) ** (n_months / 12)
    

In [12]:
def objective(trial):

    VERBOSE_PLOTS = False
    skf = StratifiedKFold(n_splits=5, shuffle=True)

    params = {
        'booster': 'gbtree',
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'n_estimators': 5000,
        'subsample': trial.suggest_float('subsample', 0.25, 0.7, log=True),
        'min_child_weight': 30,
        'early_stopping_rounds': 10,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10, log=True),
        'seed': SEED,
    }

    # Train and test model on each fold
    metrics = []
    for train_index, val_index in list(skf.split(X_train_val, y_train_val["target_xgb"]))[:3]:
        X_train, X_val = X_train_val.iloc[train_index].copy(), X_train_val.iloc[val_index].copy()
        y_train, y_val = y_train_val.iloc[train_index].copy(), y_train_val.iloc[val_index].copy()
        
        # Train model
        params['scale_pos_weight'] = (y_train["target_xgb"] == 0).sum() / (y_train["target_xgb"] == 1).sum()

        model = xgb.XGBClassifier(**params)
        model.fit(
            X_train, y_train["target_xgb"], 
            eval_set=[(X_train, y_train["target_xgb"]), (X_val, y_val["target_xgb"])],
            verbose=False
        )

        if VERBOSE_PLOTS == True:
            print(params, features)
            results = model.evals_result()
            epochs = len(results['validation_0']['auc'])
            x_axis = range(0, epochs)
            fig, ax = plt.subplots()
            ax.plot(x_axis, results['validation_0']['auc'], label='Train')
            ax.plot(x_axis, results['validation_1']['auc'], label='Test')
            ax.legend()
            plt.ylabel('AUC')
            plt.title('XGBoost AUC')
            plt.show()
    
        # Test model
        
        y_val['y_pred'] = model.predict_proba(X_val)[:,1]
        y_val['prob_for_month'] = y_val.apply(lambda x: get_proba_for_month(x.y_pred, x.term), axis=1)

        c_ind = concordance_index(
            event_times=y_val["target_month"], predicted_scores=-y_val['prob_for_month'], event_observed=y_val['target']
        )

        # print("C_INDEX", c_ind)
        # metric = roc_auc_score(y_val["target_xgb"], y_val['y_pred'])
        metrics.append(
            c_ind
        )
        print(
            "C_INDEX", round(
                c_ind, 5
            )
        )

        # if len(metrics) == 2 and max(metrics) < 0.72:
        #     break
        
    # print(
    #     f"{round(np.mean(metrics), 5)} +- {round(np.std(metrics), 5)}"
    # )

    return np.mean(metrics)

In [13]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2023-05-15 04:17:30,587] A new study created in memory with name: no-name-977483b6-0a0d-44f4-8e4d-5b3d75574cee


C_INDEX 0.68715
C_INDEX 0.68629


[I 2023-05-15 04:19:46,333] Trial 0 finished with value: 0.6871648349953875 and parameters: {'subsample': 0.2627331936320325, 'learning_rate': 0.1524645552177892, 'max_depth': 7}. Best is trial 0 with value: 0.6871648349953875.


C_INDEX 0.68805
C_INDEX 0.68926
C_INDEX 0.68725


[I 2023-05-15 04:27:02,155] Trial 1 finished with value: 0.6887082608591736 and parameters: {'subsample': 0.3649299538199618, 'learning_rate': 0.07692002043266608, 'max_depth': 4}. Best is trial 1 with value: 0.6887082608591736.


...

C_INDEX 0.69085
C_INDEX 0.69225
C_INDEX 0.69243


[I 2023-05-15 15:38:04,780] Trial 23 finished with value: 0.6915678954521466 and parameters: {'subsample': 0.6238441796628047, 'learning_rate': 0.01708041626618617, 'max_depth': 9}. Best is trial 16 with value: 0.6920631989302913.


C_INDEX 0.69003
C_INDEX 0.69033
C_INDEX 0.69201


[W 2023-05-15 17:17:45,940] Trial 24 failed with parameters: {'subsample': 0.5369865275041035, 'learning_rate': 0.01006959837507321, 'max_depth': 10} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/annakorytnik/opt/anaconda3/envs/kol_debug/lib/python3.8/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/n2/gg5n0_w95pv9ntx938ggdsx40000gn/T/ipykernel_36792/3613890265.py", line 29, in objective
    model.fit(
  File "/Users/annakorytnik/opt/anaconda3/envs/kol_debug/lib/python3.8/site-packages/xgboost/core.py", line 620, in inner_f
    return func(**kwargs)
  File "/Users/annakorytnik/opt/anaconda3/envs/kol_debug/lib/python3.8/site-packages/xgboost/sklearn.py", line 1490, in fit
    self._Booster = train(
  File "/Users/annakorytnik/opt/anaconda3/envs/kol_debug/lib/python3.8/site-packages/xgboost/core.py", line 620, in inner_f
    return func(**kwargs)
  File "/Users

KeyboardInterrupt: 

In [14]:
study.best_trial

FrozenTrial(number=16, state=TrialState.COMPLETE, values=[0.6920631989302913], datetime_start=datetime.datetime(2023, 5, 15, 9, 38, 27, 821838), datetime_complete=datetime.datetime(2023, 5, 15, 9, 51, 58, 665023), params={'subsample': 0.6647914335903077, 'learning_rate': 0.025066311555882063, 'max_depth': 9}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'subsample': FloatDistribution(high=0.7, log=True, low=0.25, step=None), 'learning_rate': FloatDistribution(high=0.2, log=True, low=0.01, step=None), 'max_depth': IntDistribution(high=10, log=True, low=3, step=1)}, trial_id=16, value=None)

[FrozenTrial(number=2, state=TrialState.COMPLETE, values=[0.5706904511659537], datetime_start=datetime.datetime(2023, 5, 13, 18, 25, 38, 19301), datetime_complete=datetime.datetime(2023, 5, 13, 18, 27, 1, 321098), params={'subsample': 0.06435314059498777, 'learning_rate': 0.036202388856127116}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'subsample': FloatDistribution(high=1.0, log=True, low=0.001, step=None), 'learning_rate': FloatDistribution(high=0.3, log=True, low=0.01, step=None)}, trial_id=2, value=None)]

[FrozenTrial(number=2, state=TrialState.COMPLETE, values=[0.5841330582073612], datetime_start=datetime.datetime(2023, 5, 13, 18, 34, 44, 410982), datetime_complete=datetime.datetime(2023, 5, 13, 18, 36, 35, 19502), params={'subsample': 0.6701497743342159, 'learning_rate': 0.18413715344103385}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'subsample': FloatDistribution(high=1.0, log=True, low=0.001, step=None), 'learning_rate': FloatDistribution(high=0.3, log=True, low=0.01, step=None)}, trial_id=2, value=None)]

FrozenTrial(number=18, state=TrialState.COMPLETE, values=[0.6288033226144973], datetime_start=datetime.datetime(2023, 5, 13, 19, 37, 31, 754674), datetime_complete=datetime.datetime(2023, 5, 13, 19, 39, 37, 640029), params={'subsample': 0.36848188910366186, 'learning_rate': 0.0314119110035605, 'features': ('annual_inc', 'loan_amnt', 'fico_average', 'emp_length')}, user_attrs={}, system_attrs={}, intermediate_values={}, 

distributions={'subsample': FloatDistribution(high=1.0, log=True, low=0.001, step=None), 'learning_rate': FloatDistribution(high=0.3, log=True, low=0.01, step=None), 'features': CategoricalDistribution(choices=(('annual_inc', 'loan_amnt'), ('annual_inc', 'loan_amnt', 'fico_average'), ('annual_inc', 'loan_amnt', 'fico_average', 'emp_length')))}, trial_id=18, value=None)

FrozenTrial(number=0, state=TrialState.COMPLETE, values=[0.6414780581888492], datetime_start=datetime.datetime(2023, 5, 13, 20, 4, 14, 72369), datetime_complete=datetime.datetime(2023, 5, 13, 20, 5, 23, 355636), params={'subsample': 0.22582945179691508, 'learning_rate': 0.014041463069293294}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'subsample': FloatDistribution(high=1.0, log=True, low=0.001, step=None), 'learning_rate': FloatDistribution(high=0.3, log=True, low=0.01, step=None)}, trial_id=0, value=None)

FrozenTrial(number=4, state=TrialState.COMPLETE, values=[0.6478027939799046], datetime_start=datetime.datetime(2023, 5, 13, 20, 43, 15, 532172), datetime_complete=datetime.datetime(2023, 5, 13, 20, 48, 4, 931532), params={'n_estimators': 297, 'subsample': 0.5023651754403868, 'learning_rate': 0.013297707588788659}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=600, log=True, low=100, step=1), 'subsample': FloatDistribution(high=1.0, log=True, low=0.01, step=None), 'learning_rate': FloatDistribution(high=0.3, log=True, low=0.01, step=None)}, trial_id=4, value=None)

FrozenTrial(number=2, state=TrialState.COMPLETE, values=[0.7056628191146204], datetime_start=datetime.datetime(2023, 5, 13, 20, 57, 13, 758039), datetime_complete=datetime.datetime(2023, 5, 13, 21, 3, 26, 599971), params={'n_estimators': 386, 'subsample': 0.5802227354327434, 'learning_rate': 0.018175234298120842}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=400, log=True, low=150, step=1), 'subsample': FloatDistribution(high=1.0, log=True, low=0.01, step=None), 'learning_rate': FloatDistribution(high=0.3, log=True, low=0.01, step=None)}, trial_id=2, value=None)

["annual_inc", "loan_amnt", "fico_average", "emp_length", "acc_open_past_24mths", "avg_cur_bal" , "sub_grade", "int_rate"]

FrozenTrial(number=2, state=TrialState.COMPLETE, values=[0.7319637194257594], datetime_start=datetime.datetime(2023, 5, 14, 20, 37, 11, 356350), datetime_complete=datetime.datetime(2023, 5, 14, 20, 40, 40, 309404), params={'subsample': 0.5036186170641379, 'learning_rate': 0.07280828054330878, 'max_depth': 8}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'subsample': FloatDistribution(high=0.6, log=True, low=0.1, step=None), 'learning_rate': FloatDistribution(high=0.2, log=True, low=0.01, step=None), 'max_depth': IntDistribution(high=10, log=True, low=3, step=1)}, trial_id=2, value=None)

features = ["annual_inc", "loan_amnt", "fico_average", "emp_length", "acc_open_past_24mths", "avg_cur_bal" , "sub_grade", "int_rate", "revol_util"]
features += [col for col in data.columns if "home_ownership_ohe_" in col]

FrozenTrial(number=4, state=TrialState.COMPLETE, values=[0.7338525205589255], datetime_start=datetime.datetime(2023, 5, 14, 21, 52, 16, 244266), datetime_complete=datetime.datetime(2023, 5, 14, 22, 1, 47, 415475), params={'subsample': 0.2619685319453447, 'learning_rate': 0.01739293623497205, 'max_depth': 10}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'subsample': FloatDistribution(high=0.7, log=True, low=0.25, step=None), 'learning_rate': FloatDistribution(high=0.2, log=True, low=0.01, step=None), 'max_depth': IntDistribution(high=10, log=True, low=3, step=1)}, trial_id=4, value=None)

features = ["annual_inc", "loan_amnt", "fico_average", "emp_length", "acc_open_past_24mths", "avg_cur_bal" , "sub_grade", "int_rate", "revol_util"]

features += [col for col in data.columns if "home_ownership_ohe_" in col]

FrozenTrial(number=16, state=TrialState.COMPLETE, values=[0.6920631989302913], datetime_start=datetime.datetime(2023, 5, 15, 9, 38, 27, 821838), datetime_complete=datetime.datetime(2023, 5, 15, 9, 51, 58, 665023), params={'subsample': 0.6647914335903077, 'learning_rate': 0.025066311555882063, 'max_depth': 9}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'subsample': FloatDistribution(high=0.7, log=True, low=0.25, step=None), 'learning_rate': FloatDistribution(high=0.2, log=True, low=0.01, step=None), 'max_depth': IntDistribution(high=10, log=True, low=3, step=1)}, trial_id=16, value=None)

# Wrong

["annual_inc", "loan_amnt", "fico_average", "emp_length", "acc_open_past_24mths", "avg_cur_bal" , "sub_grade", "dti"]

FrozenTrial(number=2, state=TrialState.COMPLETE, values=[0.7042695763788417], datetime_start=datetime.datetime(2023, 5, 13, 21, 23, 14, 409457), datetime_complete=datetime.datetime(2023, 5, 13, 21, 26, 2, 806772), params={'n_estimators': 195, 'subsample': 0.050919411040719585, 'learning_rate': 0.02744129835382001}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=500, log=True, low=150, step=1), 'subsample': FloatDistribution(high=1.0, log=True, low=0.01, step=None), 'learning_rate': FloatDistribution(high=0.3, log=True, low=0.01, step=None)}, trial_id=2, value=None)



["annual_inc", "loan_amnt", "fico_average", "emp_length", "acc_open_past_24mths", "avg_cur_bal" , "sub_grade", "initial_list_status"]

FrozenTrial(number=4, state=TrialState.COMPLETE, values=[0.7055814865786371], datetime_start=datetime.datetime(2023, 5, 13, 22, 0, 25, 924283), datetime_complete=datetime.datetime(2023, 5, 13, 22, 6, 5, 514102), params={'n_estimators': 369, 'subsample': 0.5376649220043659, 'learning_rate': 0.049542498705915004}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=500, log=True, low=150, step=1), 'subsample': FloatDistribution(high=0.6, log=True, low=0.05, step=None), 'learning_rate': FloatDistribution(high=0.3, log=True, low=0.01, step=None)}, trial_id=4, value=None)

["annual_inc", "loan_amnt", "fico_average", "emp_length", "acc_open_past_24mths", "avg_cur_bal" , "sub_grade", "initial_list_status"]